# ETL 

In [11]:
#Import libraries need for cleaning and EDA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Import libraries for modeling
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import neighbors
from sklearn import tree 
from sklearn import svm 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

In [3]:
#Reading in the data and removing the last two columns
df = pd.read_csv('BankChurners.csv')
df = df.iloc[:,1:-2]
df.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [4]:
#Seeing what kind of info/data types/ and if any nulls are present in the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Attrition_Flag            10127 non-null  object 
 1   Customer_Age              10127 non-null  int64  
 2   Gender                    10127 non-null  object 
 3   Dependent_count           10127 non-null  int64  
 4   Education_Level           10127 non-null  object 
 5   Marital_Status            10127 non-null  object 
 6   Income_Category           10127 non-null  object 
 7   Card_Category             10127 non-null  object 
 8   Months_on_book            10127 non-null  int64  
 9   Total_Relationship_Count  10127 non-null  int64  
 10  Months_Inactive_12_mon    10127 non-null  int64  
 11  Contacts_Count_12_mon     10127 non-null  int64  
 12  Credit_Limit              10127 non-null  float64
 13  Total_Revolving_Bal       10127 non-null  int64  
 14  Avg_Op

In [5]:
#Renaming the income column for easier reading 
df.Income_Category = df.Income_Category.map({'$60K - $80K':'60-80K', 'Less than $40K':'< 40K', '$80K - $120K':'80-120K', '$40K - $60K': '40-60K','$120K +':'120k>','Unknown':'Unknown'})

There seems to be "Unknown" values in the data frame. First we will see how many unknown values are present in the data frame and how we can deal with them.

In [6]:
#Checking to see what columns contain this 'Unknown value'
df[df =='Unknown'].count()

Attrition_Flag                 0
Customer_Age                   0
Gender                         0
Dependent_count                0
Education_Level             1519
Marital_Status               749
Income_Category             1112
Card_Category                  0
Months_on_book                 0
Total_Relationship_Count       0
Months_Inactive_12_mon         0
Contacts_Count_12_mon          0
Credit_Limit                   0
Total_Revolving_Bal            0
Avg_Open_To_Buy                0
Total_Amt_Chng_Q4_Q1           0
Total_Trans_Amt                0
Total_Trans_Ct                 0
Total_Ct_Chng_Q4_Q1            0
Avg_Utilization_Ratio          0
dtype: int64

Having to remove roughly 1520 values from an already small dataset would make a more imbalanced dataset, so I will change the 'unknown' values to other known values randomly.

In [7]:
#Determining how many unknown values need to be changed for each column.
print(df.Education_Level.value_counts())
print(df.Marital_Status.value_counts())
print(df.Income_Category.value_counts())

Graduate         3128
High School      2013
Unknown          1519
Uneducated       1487
College          1013
Post-Graduate     516
Doctorate         451
Name: Education_Level, dtype: int64
Married     4687
Single      3943
Unknown      749
Divorced     748
Name: Marital_Status, dtype: int64
< 40K      3561
40-60K     1790
80-120K    1535
60-80K     1402
Unknown    1112
120k>       727
Name: Income_Category, dtype: int64


In [8]:
#Using a list for each column and their values that we want to keep. 
EL = ['Graduate','High School','Uneducated','College','Post-Graduate','Doctorate']
#Using a random seed to change each unknown value to anyone in the list above
np.random.seed(1) 
m = df['Education_Level'].eq('Unknown')
df.loc[m, 'Education_Level'] = np.random.choice(EL, size=m.sum())

MS = ['Married','Single','Divorced']
   
np.random.seed(1) 
m = df['Marital_Status'].eq('Unknown')
df.loc[m, 'Marital_Status'] = np.random.choice(MS, size=m.sum())

IC = ['< 40K','40-60K','60-80K','80-120K','120k>']
   
np.random.seed(1) 
m = df['Income_Category'].eq('Unknown')
df.loc[m, 'Income_Category'] = np.random.choice(IC, size=m.sum())

In [9]:
#Checkcing to see if the unknown values are no longer here 
print(df.Education_Level.unique())
print(df.Marital_Status.unique())
print(df.Income_Category.unique())

['High School' 'Graduate' 'Uneducated' 'Doctorate' 'College'
 'Post-Graduate']
['Married' 'Single' 'Divorced']
['60-80K' '< 40K' '80-120K' '40-60K' '120k>']


In [10]:
#Exporting the current dataframe to an excel file to use in Power BI
df.to_csv('BankChurning_Updated.csv',index=False)

Taking out the unknown values and making them a random generated item does skew the results, but we can just considered this a randomly generated scenario and or case study, since if the seed number does change the results will change.